In [1]:
from pyincore import IncoreClient
from pyincore_incubator.analyses.electricpowernetworkrecovery import PowerNetworkRecoveryAnalysis
import pyincore.globals as pyglobals

import warnings
warnings.filterwarnings("ignore")

from pyincore import IncoreClient, Dataset, DataService, HazardService, FragilityService, MappingSet
from pyincore.analyses.epfdamage import EpfDamage
from pyincore.analyses.montecarlofailureprobability import MonteCarloFailureProbability

In [2]:
# client = IncoreClient(pyglobals.INCORE_API_DEV_URL)
client = IncoreClient(pyglobals.INCORE_API_PROD_URL)

data_service = DataService(client)
hazard_service = HazardService(client)
fragility_service = FragilityService(client)

Connection successful to IN-CORE services. pyIncore version detected: 1.19.0


In [3]:
# Node, edge, building, and pole damage datasets
node_dataset_id = "66912740acbe296a1b8145b3"
edge_dataset_id = "6690763bacbe296a1b8144b3"
building_dataset_id = "63ff6b135c35c0353d5ed3ac"  # Island Only


hazard_type = "hurricane"
# 100yr Hazard Event
hazard_id = "5fa5a9497e5cdf51ebf1add2"
# Number of samples
sample_num = 12
crew_num = 20

In [4]:
# EPF fragility mapping
epf_mapping_id = "62fac92ecef2881193f22613"
epf_mapping_set = MappingSet(fragility_service.get_mapping(epf_mapping_id))

epf_dmg_hurricane_galveston = EpfDamage(client)
epf_dmg_hurricane_galveston.load_remote_input_dataset("epfs", "62fc000f88470b319561b58d")
epf_dmg_hurricane_galveston.set_input_dataset('dfr3_mapping_set', epf_mapping_set)
epf_dmg_hurricane_galveston.set_parameter("result_name", "Galveston-hurricane-epf-damage")
epf_dmg_hurricane_galveston.set_parameter("fragility_key", "Non-Retrofit Fragility ID Code")
epf_dmg_hurricane_galveston.set_parameter("hazard_type", hazard_type)
epf_dmg_hurricane_galveston.set_parameter("hazard_id", hazard_id)
epf_dmg_hurricane_galveston.set_parameter("num_cpu", 8)

# Run Analysis
epf_dmg_hurricane_galveston.run_analysis()
epf_dmg_result = epf_dmg_hurricane_galveston.get_output_dataset("result")
epf_dmg_result_df = epf_dmg_result.get_dataframe_from_csv()
epf_dmg_result_df.head()

Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...


,guid,LS_0,LS_1,LS_2,LS_3,DS_0,DS_1,DS_2,DS_3,haz_expose
0,2525e200-cdc6-4fb8-9952-f17579176fd8,0.0,0.0,0.0,0.0,1.0,0,0,0.0,yes
1,9180f947-26ae-40bd-85f5-83a80525f6c8,0.0,0.0,0.0,0.0,1.0,0,0,0.0,yes
2,11ddc2dc-cc92-4c90-b4cc-a99b92a26f4b,0.0,0.0,0.0,0.0,1.0,0,0,0.0,yes
3,9e58f196-9a2c-40b1-b722-69fa9c53c9d4,0.0,0.0,0.0,0.0,1.0,0,0,0.0,yes
4,a4de2d54-b251-4a02-af0c-d12e0dc3670c,0.0,0.0,0.0,0.0,1.0,0,0,0.0,yes


In [5]:
mc = MonteCarloFailureProbability(client)
mc.set_input_dataset("damage", epf_dmg_result)
mc.set_parameter("result_name", "epf_mc")
mc.set_parameter("num_cpu", 8)
mc.set_parameter("num_samples", sample_num)
mc.set_parameter("damage_interval_keys", ["DS_0", "DS_1", "DS_2", "DS_3"])
mc.set_parameter("failure_state_keys", ["DS_1", "DS_2", "DS_3"])
mc.run_analysis()

gal_pole_damage_mcs_samples = mc.get_output_dataset('sample_failure_state')
gal_pole_damage_mcs_samples_df = gal_pole_damage_mcs_samples.get_dataframe_from_csv()
gal_pole_damage_mcs_samples_df.head()

,guid,failure
0,2525e200-cdc6-4fb8-9952-f17579176fd8,"1,1,1,1,1,1,1,1,1,1,1,1"
1,9180f947-26ae-40bd-85f5-83a80525f6c8,"1,1,1,1,1,1,1,1,1,1,1,1"
2,11ddc2dc-cc92-4c90-b4cc-a99b92a26f4b,"1,1,1,1,1,1,1,1,1,1,1,1"
3,9e58f196-9a2c-40b1-b722-69fa9c53c9d4,"1,1,1,1,1,1,1,1,1,1,1,1"
4,a4de2d54-b251-4a02-af0c-d12e0dc3670c,"1,1,1,1,1,1,1,1,1,1,1,1"


In [6]:
power_recovery_analysis = PowerNetworkRecoveryAnalysis(client)

power_recovery_analysis.load_remote_input_dataset("nodes", node_dataset_id)
power_recovery_analysis.load_remote_input_dataset("edges", edge_dataset_id)
power_recovery_analysis.load_remote_input_dataset("buildings", building_dataset_id)
power_recovery_analysis.set_input_dataset("pole_damage", gal_pole_damage_mcs_samples)
power_recovery_analysis.set_parameter("result_name", "power_analysis_output")
power_recovery_analysis.set_parameter("num_thread", 8)
power_recovery_analysis.set_parameter("num_samples", sample_num)
power_recovery_analysis.set_parameter("num_crews", crew_num)
power_recovery_analysis.set_parameter("time_step", 7)
power_recovery_analysis.set_parameter("consider_protective_devices", False)

# Run Analysis
power_recovery_analysis.run_analysis()

# Get outputs
power_test_result = power_recovery_analysis.get_output_dataset('result')
power_test_result_df = power_test_result.get_dataframe_from_csv()
power_test_result_df.head()

Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...
Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...
Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...


,guid,average_power_back_time
0,f3a21b5f-0334-4d77-911f-faa9a45c92d7,0.000000
1,80ab0289-8c72-4699-b12b-08ef2f2a5c91,9.333333
2,7916eaa8-0bfa-4b32-84ff-20f19a520f5c,4.083333
3,79008971-a534-4e45-9e40-43a2a904e59b,15.166667
4,2ad8ab6f-057f-468e-9911-6e92d9d4d01f,7.000000
